In [1]:
import time
import cv2
import pickle as pkl
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from tqdm import tqdm
from IPython import display as dp
from collections import deque
import matplotlib.animation as animation

In [2]:
def drawRectangleOnImage(img, bbox, track=0, line=[], colour=(0, 255, 0)):
    b, g, r = colour
    img = cv2.rectangle(img, (bbox[0], bbox[1]), (bbox[2], bbox[3]), (int(b), int(g), int(r)), 3)
    img = cv2.putText(img, str(track), (bbox[0], bbox[1] - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (int(b), int(g), int(r)), 3)
    for i in range(1, len(line)):
        img = cv2.line(img, line[i - 1], line[i], (int(b), int(g), int(r)), 3)
    return img

In [3]:
def get_box_center(bbox):
    center = (int(bbox[0]) + ((int(bbox[2]) - int(bbox[0])) // 2)), (int(bbox[1]) + ((int(bbox[3]) - int(bbox[1])) // 2)) # X + Width / 2, Y + Height / 2
    return center

In [4]:
# Detection Pre-Processing 

# Remove overlaps in the same frame
def remove_overlaps(detections_pd, tolerance=0.9):
    row_to_remove = []
    for detection in detections_pd.get('detection'):
        length, _ =detections_pd.shape
        for i in range(length):
            IoU = detection.IoU(detections_pd.iloc[i]['detection'])
            if IoU > tolerance and IoU < 1:
                row_to_remove.append(i)
                
    row_to_remove = np.unique(np.array(row_to_remove))
    detections_pd = detections_pd.drop(index=row_to_remove)
        
    return detections_pd

In [5]:
# Detection to DataFrame
def get_detection_dataframe(detections):
    bboxes = []
    bsizes = []
    lines = deque(maxlen=32)
    bdetections = []
    tracks = list(range(0, len(detections)))
    colours = []
    for i in range(len(detections)):
        colours.append(tuple(np.random.choice(range(256), size=3).astype('int')))
        
    updated = [False]*len(detections)
    
    for detection in detections:
        bbox = np.array(detection.getBBox()).astype('int')
        bboxes.append(bbox)
        
        centers = []
        centers.append(get_box_center(bbox))
        lines.append(centers)

        bsize = int(detection.areaOfRec())
        bsizes.append(bsize)

        bdetections.append(detection)

    detec = {
        'track': tracks,
        'detection': bdetections,
        'bbox': bboxes,
        'size': bsizes,
        'line': lines,
        'colour': colours,
        'updated': updated
    }
    detections_pd = pd.DataFrame(detec)
    detections_pd = detections_pd.sort_values(by=['size'], ascending=False)
    detections_pd = detections_pd.reset_index(drop=True)
    
    return detections_pd

In [6]:
def get_new_track_id(array):
    for cursor in range(30):
        if array[cursor] != cursor:
            return cursor
    return N

In [7]:
def update_track(detections_pd, next_detections_pd, tolerance=0.5):
    detections_pd['updated'] = False
    detections_pd = detections_pd.reset_index(drop=True)
    for index, next_detection in next_detections_pd.iterrows():
        length, _ = detections_pd.shape
        
        # Find overlaps and update if found
        for i in range(length):
            IoU = next_detection['detection'].IoU(detections_pd.iloc[i]['detection'])
            if IoU > tolerance and detections_pd['updated'].iloc[i] != True:
                detections_pd.at[i,'detection'] = next_detection['detection']
                detections_pd.at[i,'bbox'] = next_detection['bbox']
                detections_pd.at[i,'size'] = next_detection['size']
                detections_pd.at[i,'line'].append(next_detection['line'][0])
                detections_pd.at[i,'updated'] = True
                next_detections_pd.at[index, 'updated'] = True
                break
    
    # Drop detections no longer exist
    detections_pd = detections_pd[detections_pd['updated'] == True]
                
    # Start tracking new detections
    if any(next_detections_pd['updated'] == False):
        new_pd = next_detections_pd[next_detections_pd['updated']==False]
        new_pd = new_pd.reset_index(drop=True)
        
        #New Track Number for detection
        
        max_tracking_id = detections_pd['track'].max()
    
        for i in range(len(new_pd)):
            new_pd.at[i, 'track'] = max_tracking_id + 1 + i
        
        
#         counter = 0
#         for i in range(len(new_pd)):
#             while counter in detections_pd['track']:
#                 counter = counter + 1
#             new_pd.at[i, 'track'] = counter
#             counter = counter + 1

#         track_list = detections_pd['track'].tolist()
#         track_list = track_list.sort()
#         for index, new in new_pd.iterrows():
#             new_track_id = get_new_track_id(track_list)
#             new_pd.at[index, 'track'] = new_track_id
#             track_list.append(new_track_id)

        detections_pd = pd.concat([detections_pd, new_pd])
    detections_pd = detections_pd.reset_index(drop=True)
    return detections_pd

In [8]:
# Load paths
# detection_path = 'detection_pkls/retinanet_101_detections.pkl'
detection_path = 'detection_pkls/maskRCNN_101_detections.pkl'
data_path = '../datasets/AICity_data/train/S03/c010/'

# Load detections
with open(detection_path , 'rb') as f:
    all_detections = pkl.load(f)

In [9]:
# Pre-process the first frame if needed
# detections_pd = remove_overlaps(detections_pd, 0.5)
# detections_pd

In [10]:
display = False
gif = False

if gif:
    fig, ax = plt.subplots()
    plt.axis('off')

ims = []
detection_history = []

# Get the First Frame
detections_pd = get_detection_dataframe(all_detections['0'])
detection_history.append(detections_pd)

# Load video
vidcap = cv2.VideoCapture(data_path + 'vdo.avi')
_, image = vidcap.read()
num_frames = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))

# Set plot
if display:
    plt.ion() 
    plt.figure(figsize=(20, 12))
    plt.axis('off') 

# Iterate Frames
skip = 1
for frame in tqdm(range(1,num_frames//skip)):
    for i in range(skip):
        _, image = vidcap.read()
    frame = frame*skip
    
    next_detections_pd = get_detection_dataframe(all_detections[str(frame)])
    detections_pd = update_track(detections_pd, next_detections_pd, tolerance=0.5)
    detection_history.append(detections_pd)
    
    for index, row in detections_pd.iterrows():
        image = drawRectangleOnImage(image, row['bbox'], track=row['track'], line=row['line'],colour=row['colour'])
        
    if display:
        plt.title('Frame:'+str(frame))
        plt.imshow(image)  
    
    if gif:
        im = ax.imshow(image, animated=True)
        ims.append([im])
        
    if display:
        dp.clear_output(wait=True)
        dp.display(plt.gcf())
        time.sleep(0.000001)
        plt.cla()

if gif:
    ani = animation.ArtistAnimation(fig, ims, interval=10, blit=True, repeat_delay=10000)
    ani.save('Tracking_with_IoU_overlap' + ".gif", writer=animation.PillowWriter(fps=24))
    
# Open a file and use dump()
with open('tracking_history/tracking_history.pkl', 'wb') as file:
    # A new file will be created
    pkl.dump(detection_history, file)

100%|██████████| 2140/2140 [00:40<00:00, 52.86it/s]


In [11]:
with open('tracking_history/tracking_history.pkl', 'rb') as file:
    # Call load method to deserialze
    tracking_hist = pkl.load(file)

In [20]:
tracking_hist[500]

,track,detection,bbox,size,line,colour,updated
0,0,"Frame 500, TL [1287.7440185546875,358.87194824...","[1287, 358, 1520, 543]",42889,"[(1403, 451), (1405, 451), (1403, 451), (1405,...","(69, 170, 61)",False
1,4,"Frame 500, TL [1178.3533935546875,100.75473785...","[1178, 100, 1229, 160]",3075,"[(1203, 130), (1203, 129), (1203, 129), (1203,...","(47, 16, 222)",False
2,17,"Frame 500, TL [878.74658203125,92.938140869140...","[878, 92, 939, 141]",2968,"[(1298, 1038), (1308, 1014), (1308, 995), (130...","(89, 159, 205)",False
3,22,"Frame 500, TL [559.2926025390625,98.8886795043...","[559, 98, 659, 167]",6895,"[(611, 133), (611, 132), (611, 132), (611, 132...","(81, 53, 48)",False
4,25,"Frame 500, TL [923.5466918945312,77.1308517456...","[923, 77, 1015, 145]",6303,"[(983, 111), (983, 111), (981, 111), (979, 111...","(70, 151, 131)",False
5,30,"Frame 500, TL [651.4411010742188,217.087188720...","[651, 217, 807, 344]",19929,"[(666, 127), (672, 126), (675, 126), (675, 127...","(149, 20, 134)",False
6,39,"Frame 500, TL [691.3488159179688,114.424789428...","[691, 114, 793, 195]",8314,"[(671, 126), (672, 126), (673, 126), (673, 126...","(15, 44, 10)",False
7,44,"Frame 500, TL [1304.18310546875,147.3674011230...","[1304, 147, 1559, 265]",30018,"[(1563, 162), (1555, 161), (1550, 160), (1542,...","(157, 90, 17)",False
8,45,"Frame 500, TL [585.9688110351562,72.9818649291...","[585, 72, 658, 115]",3032,"[(618, 98), (620, 94), (619, 94), (620, 93), (...","(228, 169, 206)",False


In [13]:
tracking_hist[84]

,track,detection,bbox,size,line,colour,updated
0,0,"Frame 84, TL [1289.6099853515625,359.138549804...","[1289, 359, 1519, 543]",42441,"[(1403, 451), (1405, 451), (1403, 451), (1405,...","(69, 170, 61)",False
1,1,"Frame 84, TL [561.5643310546875,97.86048889160...","[561, 97, 660, 168]",6954,"[(611, 132), (611, 132), (611, 132), (610, 133...","(117, 172, 100)",False
2,5,"Frame 84, TL [924.1571655273438,77.41192626953...","[924, 77, 1015, 145]",6265,"[(969, 109), (969, 110), (969, 110), (969, 110...","(199, 112, 84)",False
3,4,"Frame 84, TL [1178.71728515625,101.22489166259...","[1178, 101, 1229, 159]",2983,"[(1203, 130), (1203, 129), (1203, 129), (1203,...","(47, 16, 222)",False
4,2,"Frame 84, TL [878.5881958007812,92.13430786132...","[878, 92, 942, 140]",3111,"[(911, 116), (910, 116), (910, 116), (912, 117...","(210, 250, 181)",False
5,3,"Frame 84, TL [586.8478393554688,74.05493164062...","[586, 74, 656, 110]",2526,"[(621, 91), (623, 92), (622, 92), (622, 91), (...","(251, 240, 144)",False
6,6,"Frame 84, TL [541.2262573242188,75.41524505615...","[541, 75, 647, 148]",7748,"[(593, 110), (594, 110), (593, 110), (594, 111...","(54, 21, 58)",False


In [14]:
# for index, row in detections_pd.iterrows():
#     print(row['colour'])
#     image = drawRectangleOnImage(image, row['bbox'], track=row['track'],colour=row['colour'])

In [15]:
# plt.figure(figsize=(20, 12))
# plt.imshow(image)

In [16]:
# #Test......
# test_num = 60
# detections_pd = get_detection_dataframe(all_detections[str(test_num)])
# next_detections_pd = get_detection_dataframe(all_detections[str(test_num+1)])

In [17]:
# detections_pd

In [18]:
# next_detections_pd

In [19]:
# detections_pd = update_track(detections_pd, next_detections_pd, tolerance=0.5)
# next_detections_pd